In [1]:
import numpy as np
import os
from os.path import join
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Activation, TimeDistributed, LSTM
from keras_utils import set_keras_session
from image_generators import frames_generator_rnn

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
set_keras_session()
dataset = 'UCF11'
nb_classes = int(dataset[3:])

In [3]:
frame_number = 50
width = 320
height = 240
channels = 3

sequence_shape = (frame_number, height, width, channels)

dataset_dir = join('datasets', dataset, ('separate_frames_{}_h_{}_w_{}_optical_padding_False').format(frame_number, height, width))

In [4]:
def build_model():
    model = Sequential()
    model.add(TimeDistributed(Conv2D(32, (5, 5), activation='relu'), input_shape=sequence_shape))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))

    model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    
    model.add(TimeDistributed(Conv2D(256, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    
    model.add(TimeDistributed(Conv2D(512, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(256, dropout=0.25))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    return model

In [25]:
batch_size = 1

train_generator = frames_generator_rnn(dataset_dir, 'train', batch_size)
valid_generator = frames_generator_rnn(dataset_dir, 'valid', batch_size)
train_step = next(train_generator)
valid_step = next(valid_generator)

In [26]:
from keras import metrics
from keras import callbacks
from functools import partial

top_3_k_categorical_accuracy = partial(metrics.top_k_categorical_accuracy, k=3)
top_3_k_categorical_accuracy.__name__ = 'top_3'

early_stopper = callbacks.EarlyStopping(patience=2)

model = build_model()
print(model.summary())
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy', top_3_k_categorical_accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_56 (TimeDis (None, 50, 236, 316, 32)  2432      
_________________________________________________________________
time_distributed_57 (TimeDis (None, 50, 118, 158, 32)  0         
_________________________________________________________________
time_distributed_58 (TimeDis (None, 50, 116, 156, 64)  18496     
_________________________________________________________________
time_distributed_59 (TimeDis (None, 50, 58, 78, 64)    0         
_________________________________________________________________
time_distributed_60 (TimeDis (None, 50, 56, 76, 128)   73856     
_________________________________________________________________
time_distributed_61 (TimeDis (None, 50, 28, 38, 128)   0         
_________________________________________________________________
time_distributed_62 (TimeDis (None, 50, 26, 36, 256)   295168    
__________

In [27]:
def train_model(**kwargs):
    return model.fit_generator(train_generator,
                    steps_per_epoch=train_step, 
                    validation_data=valid_generator,
                    validation_steps=valid_step,
                    epochs=kwargs.get('epochs', 5))

In [28]:
history = train_model(epochs=10, callbacks=[early_stopper])

Epoch 1/10
1126/1126 [==============================] - 352s 312ms/step - loss: 2.4050 - acc: 0.0888 - top_3: 0.2664 - val_loss: 2.4006 - val_acc: 0.0746 - val_top_3: 0.2452
Epoch 2/10
 314/1126 [=======>......................] - ETA: 3:17 - loss: 2.4029 - acc: 0.1178 - top_3: 0.2834

KeyboardInterrupt: 

In [ ]:
%matplotlib inline
import plot_utils

plot_utils.plot_history(history)